<a href="https://colab.research.google.com/github/Ulyssesllc/cat_dog_classification/blob/main/Cats_vs_Dogs_Classification_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
bhavikjikadara_dog_and_cat_classification_dataset_path = kagglehub.dataset_download('bhavikjikadara/dog-and-cat-classification-dataset')

print('Data source import complete.')


Data source import complete.


# I) Import and download

In [ ]:
import os
import shutil
import torch
import torchvision
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from torch import nn
from shutil import copyfile


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# II) The dataset

### *) Loading the dataset

In [ ]:
root_folder='/kaggle/input/dog-and-cat-classification-dataset/PetImages'
cat_folder= os.path.join(root_folder,"Cat")
print(cat_folder)

## II.1) Overview

In [ ]:
cat_img= os.listdir(cat_folder)
print(len(cat_img))
print(cat_img[:5])

In [ ]:
cat_img_paths= [os.path.join(cat_folder, img) for img in cat_img]
print(cat_img_paths[:5])

In [ ]:
fig, axes = plt.subplots(5,2, figsize=(10,20))
ten_img= cat_img_paths[:10]

for i, img in enumerate(ten_img):
    print(img)
    ax = axes[i//2,i%2]
    ax.imshow(plt.imread(img))
    ax.title.set_text(os.path.basename(img))
    ax.axis('on')



## II.2) Train & test split

In [ ]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/validation')
    os.mkdir('/tmp/cats-v-dogs/test')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/validation/cats')
    os.mkdir('/tmp/cats-v-dogs/validation/dogs')
    os.mkdir('/tmp/cats-v-dogs/test/cats')
    os.mkdir('/tmp/cats-v-dogs/test/dogs')
except OSError:
    print('Error failed to make directory')

****As we can see, the paths are yet to be created, so it's our mission to do that beforehand****

In [ ]:
# Define paths
cat_path = '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat'
dog_path = '/kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog'

training_path = '/tmp/cats-v-dogs/training'
validation_path = '/tmp/cats-v-dogs/validation'

training_dog = os.path.join(training_path, 'dogs/')
validation_dog = os.path.join(validation_path, 'dogs/')

training_cat = os.path.join(training_path, 'cats/')
validation_cat = os.path.join(validation_path, 'cats/')

# Define whether to test split or not
include_test= True

In [ ]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs')))

print(len(os.listdir('/tmp/cats-v-dogs/validation/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/validation/dogs')))

print(len(os.listdir('/tmp/cats-v-dogs/test/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/test/dogs')))

****Now we will create a function to split the data****

In [ ]:
def split_data(main_dir, training_dir, validation_dir, test_dir=None, include_test_split = True,  split_size=0.9):
    """
    Splits the data into train validation and test sets (optional)

    Args:
    main_dir (string):  path containing the images
    training_dir (string):  path to be used for training
    validation_dir (string):  path to be used for validation
    test_dir (string):  path to be used for test
    include_test_split (boolen):  whether to include a test split or not
    split_size (float): size of the dataset to be used for training
    """
    files = []
    for file in os.listdir(main_dir):
        if  os.path.getsize(os.path.join(main_dir, file)): # check if the file's size isn't 0
            files.append(file) # appends file name to a list

    shuffled_files = random.sample(files,  len(files)) # shuffles the data
    split = int(0.9 * len(shuffled_files)) #the training split casted into int for numeric rounding
    train = shuffled_files[:split] #training split
    split_valid_test = int(split + (len(shuffled_files)-split)/2)

    if include_test_split:
        validation = shuffled_files[split:split_valid_test] # validation split
        test = shuffled_files[split_valid_test:]
    else:
        validation = shuffled_files[split:]

    for element in train:
        copyfile(os.path.join(main_dir,  element), os.path.join(training_dir, element)) # copy files into training directory

    for element in validation:
        copyfile(os.path.join(main_dir,  element), os.path.join(validation_dir, element))# copy files into validation directory

    if include_test_split:
        for element in test:
            copyfile(os.path.join(main_dir,  element), os.path.join(test_dir, element)) # copy files into test directory
    print("Split successful!")

In [ ]:
split_data(cat_path, '/tmp/cats-v-dogs/training/cats', '/tmp/cats-v-dogs/validation/cats', '/tmp/cats-v-dogs/test/cats',include_test, 0.9)
split_data(dog_path, '/tmp/cats-v-dogs/training/dogs', '/tmp/cats-v-dogs/validation/dogs','/tmp/cats-v-dogs/test/dogs',include_test, 0.9)

****Now, lets check on the number of files stored in each recently created directories****

In [ ]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs')))

print(len(os.listdir('/tmp/cats-v-dogs/validation/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/validation/dogs')))


print(len(os.listdir('/tmp/cats-v-dogs/test/cats')))
print(len(os.listdir('/tmp/cats-v-dogs/test/dogs')))

## II.3) Creation of image loaders

****First, we define the transformation****

In [ ]:
from torchvision import transforms, datasets
transform = transforms.Compose([transforms.Resize((150,150)),transforms.ToTensor()])

****Then, we create data loaders for previewing the images****

In [ ]:
from torch.utils.data import DataLoader
# Load datasets
train_dataset = datasets.ImageFolder(root='/tmp/cats-v-dogs/training', transform=transform)
validation_dataset = datasets.ImageFolder(root='/tmp/cats-v-dogs/validation', transform=transform)

if include_test:
    test_dataset = datasets.ImageFolder(root='/tmp/cats-v-dogs/validation', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)

if include_test:
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

****Now, make sure we got the correct data****

In [ ]:
class_names = ['Cat', 'Dog']

def plot_data(data_loader, n_images):
    """
    Plots random data from a PyTorch DataLoader
    Args:
        data_loader: a PyTorch DataLoader instance
        n_images: number of images to plot
    """
    # Fetch a batch of images and labels
    images, labels = next(iter(data_loader))  # Use iter() and next() to get a batch

    # Calculate the number of rows and columns for subplots
    n_cols = 3  # Number of columns in the plot
    n_rows = (n_images + n_cols - 1) // n_cols  # Calculate rows dynamically

    plt.figure(figsize=(14, 15))

    for i in range(n_images):
        plt.subplot(n_rows, n_cols, i + 1)
        image = images[i].permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for matplotlib
        if image.shape[-1] == 1:  # Grayscale image
            plt.imshow(image.squeeze(), cmap='gray')
        else:  # RGB image
            plt.imshow(image)
        plt.title(class_names[labels[i].item()])  # Get the label as a Python scalar
        plt.axis('off')

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

In [ ]:
plot_data(train_loader, n_images=6)

In [ ]:
plot_data(validation_loader, n_images=6)

In [ ]:
if include_test:
    plot_data(test_loader, n_images=9)

# III) Model

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional layers with BatchNorm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # (64, 75, 75)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # (128, 37, 37)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)

        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # (256, 18, 18)

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers
        self.fc1 = nn.Linear(256, 512)
        self.dropout = nn.Dropout(0.5)  # Dropout for regularization
        self.fc2 = nn.Linear(512, 2)  # Binary classification

    def forward(self, x):
        # Conv + BatchNorm + ReLU6
        x = F.relu6(self.bn1(self.conv1(x)))
        x = F.relu6(self.bn2(self.conv2(x)))
        x = self.pool1(x)

        x = F.relu6(self.bn3(self.conv3(x)))
        x = F.relu6(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = F.relu6(self.bn5(self.conv5(x)))
        x = F.relu6(self.bn6(self.conv6(x)))
        x = self.pool3(x)

        # Global Average Pooling
        x = self.global_avg_pool(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Fully connected layers
        x = F.relu6(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)

        return x  # No softmax, use nn.CrossEntropyLoss()

# Instantiate the model
model = CNN()
model = model.to(device)


****Now we define the loss function and optimizer used for the model****

In [ ]:
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer (in this case Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# IV) Evaluation

****After all those implementations and setups, we are ready to run the model and present the results****

In [ ]:
# Initialize lists to store training and validation metrics
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # Load the data to the current device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print statistics
        running_loss += loss.item()

    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    # Validation loop (optional)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # Disable gradient computation
        for images, labels in validation_loader:

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item()

    val_loss /= len(validation_loader)
    val_acc = 100 * val_correct / val_total
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

****Now we make evaluations on the test data****

In [ ]:
if include_test:
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():  # Disable gradient computation
        for images, labels in test_loader:

            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            # Accumulate loss
            test_loss += loss.item()

    # Compute average loss and accuracy
    test_loss /= len(test_loader)
    test_acc = 100 * test_correct / test_total

    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

## IV.1) Visualize the prediction

In [ ]:
def plot_prediction(data_loader, model, n_images, class_names):
    """
    Test the model on random predictions
    Args:
        data_loader: PyTorch DataLoader instance
        model: Trained PyTorch model
        n_images: Number of images to plot
        class_names: List of class names (e.g., ['Cat', 'Dog'])
    """
    model.eval()  # Set the model to evaluation mode
    images, labels = next(iter(data_loader))  # Fetch a batch of images and labels

    # Move images and labels to the appropriate device (e.g., GPU if available)
    device = next(model.parameters()).device
    images = images.to(device)
    labels = labels.to(device)

    # Get model predictions
    with torch.no_grad():
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)  # Get the predicted class indices

    # Convert tensors to numpy arrays for visualization
    images = images.cpu().numpy()
    labels = labels.cpu().numpy()
    predictions = predictions.cpu().numpy()

    # Plot the images with predictions
    plt.figure(figsize=(14, 15))
    for i in range(min(n_images, len(images))):  # Ensure we don't exceed the batch size
        plt.subplot(4, 3, i + 1)
        image = np.transpose(images[i], (1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
        if images[i].shape[0] == 1:  # Grayscale image
            image = image.squeeze()
            plt.imshow(image, cmap='gray')
        else:  # RGB image
            plt.imshow(image)

        # Set title color based on prediction correctness
        if predictions[i] == labels[i]:
            title_obj = plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[predictions[i]]}", color='g')
        else:
            title_obj = plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[predictions[i]]}", color='r')
        plt.axis('off')

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

In [ ]:
plot_prediction(validation_loader, model, n_images=9, class_names=['Cat','Dog'])

In [ ]:
if include_test:
    plot_prediction(test_loader, model, n_images=9, class_names=['Cat','Dog'])

## IV.2) Visualize training process

In [ ]:
# Create a DataFrame to store the training history
results = pd.DataFrame({
    'epoch': range(1, num_epochs + 1),
    'train_loss': train_loss_history,
    'train_acc': train_acc_history,
    'val_loss': val_loss_history,
    'val_acc': val_acc_history
})

# Display the last few rows of the DataFrame
print(results.tail())

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(results['epoch'], results['train_loss'], label='Train Loss')
plt.plot(results['epoch'], results['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(results['epoch'], results['train_acc'], label='Train Accuracy')
plt.plot(results['epoch'], results['val_acc'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
